# Dynamic Data Masking

Dynamische Datenmaskierung versteckt die tatsächlichen Daten hinter einer Maske von Werten, die anstelle der Daten angezeigt werden.

Die tatsächlichen Daten können nur von Datenbank-Usern mit der Berechtigung **UNMASK** eingesehen werden.

  

Folgende Maskierungsfunktionen stehen zur Verfügung:

  

- **default()** zeigt XXXX anstelle der eigentlichen Werte an.
- **email()** zeigt den ersten Buchstaben der E\_Mail Adresse an und maskiert den Rest, z. Bsp. aXXX@XXXX.com
- **random(von, bis)** zeigt eine Zufallszahl im angegebenen Bereich an
- **partial()** ermöglicht die teilweise Maskierung der Werte mit Anzeige einiger Zeichen des eigentlichen Wertes am Anfang und/oder Ende.
- SQL Server 2022 bringt eine neue Funktion zur Maskierung von Datumswerten hinzu: **datetime()**

[https://learn.microsoft.com/en-us/sql/relational-databases/security/dynamic-data-masking?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/relational-databases/security/dynamic-data-masking?view=sql-server-ver16)

In [ ]:
USE SQL_Seminar;
GO
-- schema to contain user tables
CREATE SCHEMA Data;
GO

-- table with masked columns
CREATE TABLE Data.Membership(
    MemberID        int IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
    FirstName        varchar(100) MASKED WITH (FUNCTION = 'partial(1, "xxxxx", 1)') NULL,
    LastName        varchar(100) NOT NULL,
    Phone            varchar(12) MASKED WITH (FUNCTION = 'default()') NULL,
    Email            varchar(100) MASKED WITH (FUNCTION = 'email()') NOT NULL,
    DiscountCode    smallint MASKED WITH (FUNCTION = 'random(1, 100)') NULL
    );

-- inserting sample data
INSERT INTO Data.Membership (FirstName, LastName, Phone, Email, DiscountCode)
VALUES   
('Roberto', 'Tamburello', '555.123.4567', 'RTamburello@contoso.com', 10),  
('Janice', 'Galvin', '555.123.4568', 'JGalvin@contoso.com.co', 5),  
('Shakti', 'Menon', '555.123.4570', 'SMenon@contoso.net', 50),  
('Zheng', 'Mu', '555.123.4569', 'ZMu@contoso.net', 40);  

In [11]:
CREATE USER MaskingTestUser WITHOUT LOGIN;  

GRANT SELECT ON SCHEMA::Data TO MaskingTestUser;  
  
  -- impersonate for testing:
EXECUTE AS USER = 'MaskingTestUser';  

SELECT * FROM Data.Membership;  

REVERT;

(4 rows affected)

Total execution time: 00:00:00.026

MemberID,FirstName,LastName,Phone,Email,DiscountCode
1,Rxxxxxo,Tamburello,xxxx,RXXX@XXXX.com,20
2,Jxxxxxe,Galvin,xxxx,JXXX@XXXX.com,14
3,Sxxxxxi,Menon,xxxx,SXXX@XXXX.com,38
4,Zxxxxxg,Mu,xxxx,ZXXX@XXXX.com,11


In [ ]:
ALTER TABLE Data.Membership  
ALTER COLUMN LastName ADD MASKED WITH (FUNCTION = 'partial(2,"xxxx",0)');

In [ ]:
ALTER TABLE Data.Membership  
ALTER COLUMN LastName varchar(100) MASKED WITH (FUNCTION = 'default()');

In [ ]:
GRANT UNMASK TO MaskingTestUser;  

EXECUTE AS USER = 'MaskingTestUser';  

SELECT * FROM Data.Membership;  

REVERT;    
  
-- Removing the UNMASK permission  
REVOKE UNMASK TO MaskingTestUser;  

In [ ]:
ALTER TABLE Data.Membership   
ALTER COLUMN LastName DROP MASKED;

In [ ]:
-- Aufräumen
DROP USER MaskingTestUser;

DROP TABLE Data.Membership;

DROP Schema Data;

## Weitere Demos mit feineren Einstellungen der Berechtigungen

[https://learn.microsoft.com/en-us/sql/relational-databases/security/dynamic-data-masking?view=sql-server-ver16#granular-permission-examples](https://learn.microsoft.com/en-us/sql/relational-databases/security/dynamic-data-masking?view=sql-server-ver16#granular-permission-examples)